In [1]:
import re
import mysql.connector
import xml.etree.ElementTree as ET

In [2]:
mydb = mysql.connector.connect(
    host='127.0.0.1', 
    user='Yuxin Xiao', 
    password='19961002XYX',
    database='CS510Project')
mycursor = mydb.cursor()

In [38]:
mycursor.execute("CREATE TABLE Sentences (QA_id INT, Subject_id INT, Sentence_id INT, Sentences TEXT)")
mycursor.execute("CREATE TABLE Tags (QA_id INT, Category VARCHAR(255), Main_category VARCHAR(255), Subcategory VARCHAR(255))")
mycursor.execute("CREATE TABLE Words (Word VARCHAR(255), Count INT, Inverted_index LONGTEXT)")

In [7]:
tree = ET.parse('data/manner.xml')
root = tree.getroot()

In [8]:
regex = r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s"
subst = "\\n"

In [39]:
def insert_sentences(QA_id, QA_pair):
    sql = ('INSERT INTO Sentences '
           '(QA_id, Subject_id, Sentence_id, Sentences) '
           'VALUES (%s, %s, %s, %s)')
    
    question = QA_pair[0].findall('subject')[0].text
    question = re.sub(regex, subst, question, 0, re.MULTILINE).split('\n')
    question = [sentence.lstrip().replace('<br />', '') for sentence in question]
    question = list(filter(None, question))
    for sentence_id, sentence in enumerate(question):
        val = (QA_id+1, 0, sentence_id+1, sentence)
        mycursor.execute(sql, val)
    
    
    answers = QA_pair[0].findall('nbestanswers')[0]
    for answer_id, answer in enumerate(answers):
        answer = re.sub(regex, subst, answer.text, 0, re.MULTILINE).split('\n')
        answer = [sentence.lstrip().replace('<br />', '') for sentence in answer]
        answer = list(filter(None, answer))
        
        for sentence_id, sentence in enumerate(answer):
            val = (QA_id+1, answer_id+1, sentence_id+1, sentence)
            mycursor.execute(sql, val)

In [40]:
def insert_tag(QA_id, QA_pair):
    def extract_tags(category_name):
        tags = QA_pair[0].findall(category_name)
        if not tags:
            return None
        tags = tags[0].text
        tags = re.sub(regex, subst, tags, 0, re.MULTILINE).split('&')
        tags = [tag.lstrip().rstrip() for tag in tags]
        tags = ', '.join(tags)
        return tags
    cat = extract_tags('cat')
    maincat = extract_tags('maincat')
    subcat = extract_tags('subcat')
    sql = ('INSERT INTO Tags '
           '(QA_id, Category, Main_category, Subcategory) '
           'VALUES (%s, %s, %s, %s)')
    val = (QA_id+1, cat, maincat, subcat)
    mycursor.execute(sql, val)

In [41]:
for QA_id, QA_pair in enumerate(root):
    insert_sentences(QA_id, QA_pair)
    insert_tag(QA_id, QA_pair)
    if QA_id % 5000 == 0:
        print(QA_id)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000


In [43]:
mydb.commit()

In [44]:
mycursor.execute("SELECT Sentences FROM Sentences")
myresult = mycursor.fetchall()
with open('All Sentences.txt','w') as file:
    for sentence in myresult:
        sentence = str(sentence).encode('utf-8','ignored')
        file.write("%s\n" % sentence)

In [3]:
mycursor.execute("SELECT * FROM Sentences")
myresult = mycursor.fetchall()

In [3]:
mycursor.execute("DROP TABLE Words")
mycursor.execute("CREATE TABLE Words (Word VARCHAR(255), Count INT, Inverted_index LONGTEXT)")

In [ ]:
sentences_count = len(myresult)
epoch_count = int(sentences_count / 10000 + 1)
print(epoch_count)

for epoch in range(epoch_count):
    
    print(epoch)
    
    count_dict = {}
    index_dict = {}
    
    for row in myresult[:(epoch+1)*10000]:
        QA_id, subject_id, sentence_id = row[0], row[1], row[2]
        sentence = re.sub(r'[^\w\s]', "", row[3]).split()
        index = "%d-%d-%d" % (QA_id, subject_id, sentence_id)
        
        for word in sentence:
            if word.isdigit():
                continue
            word = word.lower()
            
            if word in count_dict:
                count_dict[word] += 1
                index_dict[word] += (', ' + index)
            else:
                count_dict[word] = 1
                index_dict[word] = index
        
    for word in count_dict.keys():
        count = count_dict[word]
        index = index_dict[word]
        
        select_sql = "SELECT * FROM Words WHERE Word = %s"        
        mycursor.execute(select_sql, (word,))
        corr_row = mycursor.fetchall()
        
        if not corr_row:
            insert_sql = "INSERT INTO Words (Word, Count, Inverted_index) VALUES (%s, %s, %s)"
            insert_val = (word, count, index)
            mycursor.execute(insert_sql, insert_val)
        else:
            update_count = corr_row[0][1] + count
            update_index = corr_row[0][2] + ", " + index
            update_sql = "UPDATE Words SET Count = %s, Inverted_index = %s WHERE Word = %s"
            update_val = (update_count, update_index, word)
            mycursor.execute(update_sql, update_val)

322
0
1
2
3
4


In [19]:
mycursor.execute("SELECT * FROM Words")
test = mycursor.fetchall()
for x in test:
    print(x)

('how', 3078, '1-0-1, 1-1-5, 2-0-1, 3-0-1, 4-0-1, 5-0-1, 6-0-1, 6-1-1, 6-3-2, 7-0-1, 8-0-1, 9-0-1, 10-0-1, 11-0-1, 12-0-1, 13-0-1, 14-0-1, 15-0-1, 16-0-1, 17-0-1, 18-0-1, 19-0-1, 20-0-1, 21-0-1, 22-0-1, 23-0-1, 24-0-1, 25-0-1, 26-0-1, 27-0-1, 28-0-1, 29-0-1, 30-0-1, 31-0-1, 32-0-1, 33-0-1, 34-0-1, 35-0-1, 36-0-1, 37-0-1, 38-0-1, 38-2-1, 39-0-1, 40-0-1, 41-0-1, 42-0-1, 43-0-1, 44-0-1, 45-0-1, 46-0-1, 47-0-1, 48-0-1, 49-0-1, 50-0-1, 51-0-1, 52-0-1, 53-0-1, 54-0-1, 55-0-1, 55-2-4, 55-2-4, 55-3-3, 55-3-13, 56-0-1, 57-0-1, 58-0-1, 58-1-2, 59-0-1, 60-0-1, 60-1-3, 61-0-1, 62-0-1, 63-0-1, 64-0-1, 64-1-6, 65-0-1, 66-0-1, 67-0-1, 68-0-1, 69-0-1, 70-0-1, 71-0-1, 72-0-1, 73-0-1, 74-0-1, 75-0-1, 76-0-1, 76-1-8, 77-0-1, 78-0-1, 78-3-1, 79-0-1, 80-0-1, 81-0-1, 82-0-1, 83-0-1, 84-0-1, 85-0-1, 86-0-1, 87-0-1, 88-0-1, 88-1-23, 89-0-1, 89-1-15, 89-1-15, 90-0-1, 91-0-1, 92-0-1, 93-0-1, 94-0-1, 95-0-1, 96-0-1, 97-0-1, 98-0-1, 99-0-1, 100-0-1, 101-0-1, 102-0-1, 102-1-12, 103-0-1, 103-1-18, 104-0-1, 105-0-1,

('rolling', 10, '38-2-8, 65-2-4, 96-2-3, 146-0-1, 146-1-4, 38-2-8, 65-2-4, 96-2-3, 146-0-1, 146-1-4')
('lower', 39, '38-2-9, 60-1-7, 69-2-2, 81-2-8, 116-2-10, 139-1-2, 140-1-3, 183-2-3, 196-0-1, 196-3-1, 231-1-6, 262-2-6, 348-2-19, 348-2-32, 38-2-9, 60-1-7, 69-2-2, 81-2-8, 116-2-10, 139-1-2, 140-1-3, 183-2-3, 196-0-1, 196-3-1, 231-1-6, 262-2-6, 348-2-19, 348-2-32, 362-2-2, 378-1-9, 495-1-13, 519-1-3, 577-3-2, 670-1-18, 670-1-29, 674-1-16, 724-2-2, 754-2-20, 780-1-2')
('simmer', 6, '38-2-10, 43-1-12, 262-2-6, 38-2-10, 43-1-12, 262-2-6')
('bright', 5, '38-2-10, 49-1-16, 38-2-10, 49-1-16, 553-1-2')
('red', 42, '38-2-10, 39-1-9, 39-1-24, 39-1-28, 39-1-32, 105-14-19, 160-1-5, 165-1-1, 165-1-3, 203-1-2, 205-1-5, 205-1-5, 205-1-6, 275-1-1, 275-1-2, 353-2-2, 353-2-5, 38-2-10, 39-1-9, 39-1-24, 39-1-28, 39-1-32, 105-14-19, 160-1-5, 165-1-1, 165-1-3, 203-1-2, 205-1-5, 205-1-5, 205-1-6, 275-1-1, 275-1-2, 353-2-2, 353-2-5, 485-1-3, 563-0-1, 577-1-4, 577-4-20, 636-0-1, 636-1-1, 636-1-2, 659-1-2')
('

('disappears', 2, '73-1-3, 73-1-3')
('decreased', 4, '73-1-3, 222-1-32, 73-1-3, 222-1-32')
('shining', 4, '73-1-4, 73-1-5, 73-1-4, 73-1-5')
('due', 17, '73-1-5, 82-2-1, 170-1-6, 241-1-32, 344-2-7, 73-1-5, 82-2-1, 170-1-6, 241-1-32, 344-2-7, 511-1-4, 564-1-2, 649-1-3, 660-1-9, 668-3-3, 720-1-4, 749-0-1')
('friction', 4, '73-1-5, 73-1-5, 73-1-5, 73-1-5')
('resists', 2, '73-1-5, 73-1-5')
('moves', 8, '73-1-5, 168-2-2, 168-2-3, 202-2-3, 73-1-5, 168-2-2, 168-2-3, 202-2-3')
('allways', 5, '73-1-6, 73-1-6, 730-2-4, 755-2-1, 755-2-1')
('direction', 20, '73-1-6, 73-1-8, 188-2-1, 268-4-4, 327-2-32, 344-2-3, 344-2-4, 344-2-4, 344-2-5, 73-1-6, 73-1-8, 188-2-1, 268-4-4, 327-2-32, 344-2-3, 344-2-4, 344-2-4, 344-2-5, 436-1-6, 534-1-7')
('habbit', 2, '73-1-7, 73-1-7')
('hiding', 2, '73-1-7, 73-1-7')
('batsman', 6, '73-1-7, 73-1-8, 73-2-3, 73-1-7, 73-1-8, 73-2-3')
('delivery', 8, '73-1-7, 108-1-8, 108-1-9, 108-1-10, 73-1-7, 108-1-8, 108-1-9, 108-1-10')
('guessing', 2, '73-1-8, 73-1-8')
('turbulent', 4,

('httpwwwsourceforgecom', 2, '115-1-5, 115-1-5')
('whilch', 2, '115-1-5, 115-1-5')
('gpl', 2, '115-1-5, 115-1-5')
('licence', 5, '115-1-5, 194-0-1, 115-1-5, 194-0-1, 633-0-2')
('study', 25, '115-1-5, 141-0-1, 115-1-5, 141-0-1, 378-1-12, 379-0-1, 379-1-6, 379-2-1, 387-1-16, 387-1-25, 387-1-26, 394-1-6, 408-1-6, 408-2-1, 417-1-4, 470-1-11, 487-1-1, 487-3-2, 487-3-3, 670-1-16, 701-3-3, 701-5-2, 717-1-1, 719-1-4, 780-1-3')
('modify', 7, '115-1-5, 273-2-1, 115-1-5, 273-2-1, 614-2-2, 638-1-1, 741-3-1')
('freely', 2, '115-1-5, 115-1-5')
('httpwwwozekihu', 2, '115-1-6, 115-1-6')
('httpwwwcellularcozasms_servershtm', 2, '115-1-7, 115-1-7')
('httpwwwsoft32comdownload_27574html', 2, '115-1-8, 115-1-8')
('theek', 2, '115-2-1, 115-2-1')
('anirudh', 2, '115-2-1, 115-2-1')
('goodluck', 4, '115-2-2, 115-2-2, 423-2-10, 694-2-9')
('address', 65, '116-0-1, 116-2-1, 116-2-3, 116-2-5, 116-2-6, 116-2-12, 116-2-18, 163-3-7, 167-0-1, 167-0-1, 167-1-2, 167-1-2, 167-2-1, 186-1-1, 187-2-2, 189-2-1, 203-1-2, 219-

('pinesol', 2, '188-2-1, 188-2-1')
('walmartand', 2, '188-2-1, 188-2-1')
('label', 14, '188-3-2, 208-1-2, 215-1-2, 229-1-4, 229-1-7, 229-1-19, 188-3-2, 208-1-2, 215-1-2, 229-1-4, 229-1-7, 229-1-19, 431-1-6, 636-1-11')
('maps', 8, '189-0-1, 344-2-10, 344-2-11, 189-0-1, 344-2-10, 344-2-11, 365-1-2, 661-1-8')
('httpmapsyahoocom', 2, '189-1-1, 189-1-1')
('goand', 2, '189-2-1, 189-2-1')
('homework', 6, '190-1-1, 241-1-2, 190-1-1, 241-1-2, 531-1-2, 667-1-1')
('housework', 2, '190-1-4, 190-1-4')
('childcare', 2, '190-1-4, 190-1-4')
('putz', 2, '190-2-4, 190-2-4')
('understands', 5, '190-2-4, 327-2-22, 190-2-4, 327-2-22, 518-2-6')
('loves', 9, '190-2-6, 332-5-4, 190-2-6, 332-5-4, 450-1-4, 450-1-6, 518-2-6, 666-2-1, 689-2-1')
('deserving', 2, '190-2-6, 190-2-6')
('carzy', 2, '190-2-8, 190-2-8')
('hectic', 2, '190-2-8, 190-2-8')
('weekend', 3, '190-2-8, 190-2-8, 451-2-2')
('butif', 2, '190-2-9, 190-2-9')
('balloon', 2, '191-0-1, 191-0-1')
('greatly', 4, '191-1-1, 327-1-2, 191-1-1, 327-1-2')
('ap

('rehabilitation', 6, '291-1-7, 291-1-7, 291-1-8, 291-1-7, 291-1-7, 291-1-8')
('referral', 2, '291-1-8, 291-1-8')
('ideally', 5, '291-1-10, 328-1-1, 291-1-10, 328-1-1, 617-2-2')
('independance', 2, '291-1-10, 291-1-10')
('schizophrenic', 4, '291-1-12, 291-2-10, 291-1-12, 291-2-10')
('childrenthese', 2, '291-1-12, 291-1-12')
('assistance', 3, '291-1-13, 291-1-13, 705-4-1')
('reccomend', 3, '291-2-1, 291-2-1, 776-1-3')
('investigating', 2, '291-2-1, 291-2-1')
('peer', 2, '291-2-1, 291-2-1')
('chronic', 2, '291-2-1, 291-2-1')
('lossof', 2, '291-2-2, 291-2-2')
('personallity', 2, '291-2-2, 291-2-2')
('flaw', 2, '291-2-2, 291-2-2')
('improper', 2, '291-2-2, 291-2-2')
('hormones', 2, '291-2-2, 291-2-2')
('schizoaffective', 4, '291-2-3, 291-2-10, 291-2-3, 291-2-10')
('repsond', 2, '291-2-4, 291-2-4')
('dosage', 6, '291-2-4, 291-2-4, 570-1-1, 570-1-3, 570-1-3, 570-1-3')
('diffiuclt', 2, '291-2-5, 291-2-5')
('msot', 2, '291-2-5, 291-2-5')
('reopen', 2, '291-2-6, 291-2-6')
('cannabis', 8, '291-2

('serves', 1, '392-1-5')
('gears', 2, '392-1-5, 392-1-7')
('sump', 1, '392-1-5')
('abrassive', 1, '392-1-5')
('bearings', 1, '392-1-6')
('bushings', 1, '392-1-6')
('hp', 3, '392-1-7, 392-1-7, 741-1-2')
('thinit', 1, '392-1-7')
('slipprobably', 1, '392-1-7')
('3rd', 1, '392-1-7')
('creating', 5, '393-1-1, 584-1-1, 611-1-2, 673-3-2, 701-1-2')
('primer', 1, '393-1-2')
('httppodcastsyahoocompublish', 1, '393-1-3')
('theatre', 6, '394-1-2, 394-1-4, 394-1-4, 394-1-14, 394-2-2, 394-2-3')
('sinc', 1, '394-1-2')
('ethe', 1, '394-1-2')
('internshipsapprenticeships', 1, '394-1-4')
('employable', 1, '394-1-4')
('springboards', 1, '394-1-5')
('ranks', 1, '394-1-5')
('wastye', 1, '394-1-11')
('equity', 4, '394-1-11, 791-1-9, 791-1-10, 791-2-2')
('theatrical', 1, '394-1-11')
('roughly', 2, '394-1-12, 670-1-21')
('unles', 1, '394-1-12')
('aftrasag', 1, '394-1-12')
('leap', 2, '394-1-13, 508-1-2')
('pond', 1, '394-1-13')
('avital', 1, '394-1-14')
('filmcommercial', 1, '394-1-14')
('crazed', 1, '394-1-1

('vingear', 1, '533-1-1')
('vinegear', 1, '533-1-2')
('thogh', 1, '533-1-2')
('becareful', 2, '533-2-1, 764-1-11')
('believ', 1, '533-2-1')
('neutrilzer', 1, '533-2-1')
('poisons', 1, '533-2-1')
('sewer', 1, '534-0-1')
('layout', 2, '534-0-1, 761-1-27')
('measurements', 5, '534-1-1, 566-1-21, 593-1-7, 593-1-9, 621-0-1')
('sketch', 1, '534-1-1')
('septic', 2, '534-1-3, 534-1-8')
('reinforcing', 1, '534-1-5')
('5lb', 1, '534-1-5')
('davo', 3, '534-2-1, 534-2-1, 534-2-1')
('smilin', 1, '535-1-1')
('flirts', 1, '535-2-2')
('bragging', 1, '535-2-2')
('shd', 1, '536-1-1')
('ving', 1, '536-1-1')
('depicting', 1, '537-0-1')
('lord', 4, '537-0-1, 661-0-1, 661-2-2, 719-1-1')
('flies', 1, '537-0-1')
('buyrentborrow', 1, '537-1-1')
('httpwwwamazoncomgpproductb00005o06xqid1134260457sr11refsr_1_100204169915500021sdvdvglancen130', 1, '537-1-2')
('httpimagesgooglecom', 1, '537-1-5')
('httpimagessearchyahoocom', 1, '537-1-6')
('hunting', 2, '537-1-7, 765-1-2')
('likeness', 1, '537-2-1')
('depict', 1, '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
mydb.commit()

In [ ]:
mycursor.close()
mydb.close()

In [37]:
mycursor.execute("DROP TABLE Sentences")
mycursor.execute("DROP TABLE Tags")
mycursor.execute("DROP TABLE Words")

In [45]:
nonletters = '^[^a-zA-z]*|[^a-zA-Z]*$'
with open('All Sentences.txt', 'r') as file:
    sentences = file.read().split('\nb')
    sentences[0] = sentences[0][2:]
    sentences = [re.sub(nonletters, '', sentence[:-2].replace('\\','')) for sentence in sentences]